In [1274]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
data_location = Path("../Data/Tableau_Terra_workspace")

# Create the terra workspace 
Read all these weird files and try to merge them to the best of my capabilities:

1. Read and merge CLF with Premium
2. Read and merge Panel data
3. Read RNA data
4. Read and merge WES data - UPCOMMING
5. Apply loop through rows
6. Duplicate rows and add conditions
7. Check number of datapoints

Note, losing alot of information due to inconsistencies in the datasets. Another note this workflow eats up memory, because I keep meta data open, just something to note

### 1 Read and merge CLF with Premium

In [1152]:
#CLF data and cleaning
CLF_data_meta = pd.read_csv(data_location / "CLF_Cohort_Summary_full_data.csv", encoding = 'utf-8', sep='\t')
CLF_data = CLF_data_meta[["Cell Line", "PT-ID", "SM-ID", "Status", "Resolution", "T/N", "Primary Disease", "Tissue site", "Tumor type", "CELL_LINE_ID(raw)", "Media type", "Growth Properties", "Flask coating", "Starting Material Type", "Fail Mode"]]
CLF_data = CLF_data.replace(r'^\s*$', np.nan, regex=True)
CLF_data = CLF_data.rename(columns={"Cell Line": "Cell Line Starting Sample", "Tissue site": "Tissue Site", "Tumor type": "Tumor Type", "CELL_LINE_ID(raw)":"Cell Line", "Growth Properties": "Dimension"})
CLF_data["Dataset"] = "CLF"

In [1153]:
#Premium data and cleaning
Premium_data_meta = pd.read_csv(data_location / "Premium_Cohort_Summary_full_data.csv", encoding = 'utf-8', sep='\t')
Premium_data = Premium_data_meta[["Cell Line Starting Sample", "PT_ID", "Passage Sample", "JIRA Status", "Resolution", "T/N", "Primary disease", "Tissue Site", "Tumor Type", "CELL_LINE_ID", "Growth Medium", "Growth Pattern", "ECM Bio Coatings", "Root Material Type", "Passage Sample Status"]]
Premium_data = Premium_data.replace(r'^\s*$', np.nan, regex=True)
Premium_data = Premium_data.rename(columns={"PT_ID": "PT-ID", "Passage Sample": "SM-ID", "JIRA Status": "Status", "Primary disease": "Primary Disease", "CELL_LINE_ID":"Cell Line", "Growth Medium":"Media type", "Growth Pattern":"Dimension", "ECM Bio Coatings":"Flask coating", "Root Material Type":"Starting Material Type", "Passage Sample Status":"Fail Mode"})
Premium_data["Dataset"] = "Premium"

/opt/miniconda3/envs/Broad_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (5,33,80,95) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [1154]:
# Merge CLF and Premium
data = pd.concat([CLF_data, Premium_data])

# Remove Normals
data = data[data["T/N"] == "Tumor"]

# Cleanup some of columns
data["SM-ID"] = [re.split('\r|  ',x)[0].strip(' \t\n\r') if str(x) != 'nan' else x for x in list(data["SM-ID"])]
data["Resolution"] = [str(x) for x in list(data["Resolution"])]
data["Cell Line"] = [str(x) for x in list(data["Cell Line"])]
data["Media type"] = [str(x) for x in list(data["Media type"])]
data = data[data["Media type"] != "nan"]

# Note Dropping lot of conditions
Resolution_dict = {
    'Terminated':'Terminated',
    'In Process':'In Process',
    'Verification':'In Process',
    'Ambiguous':'Drop',
    'Completed':'Completed',
    'Verified Normal':'Verified Normal',
    'Verified Tumor':'Verified Tumor',
    'Clinical team follow up':'In Process',
    'Other verified tumor models':'Verified Tumor',
    'Primary sample verified normal':'Verified Normal',
    'Blacklisted; FP Failure':'Drop',
    'Low Purity Tumor':'Drop',
    'Clinical team follow up,Primary sample verified normal':'Verified Normal',
    'Clinical team follow up,Other verified tumor models':'Verified Tumor',
    'nan':'nan',
    'Abandoned':'Drop',
}

data["Resolution_modified"] = data["Resolution"]
data = data.replace({"Resolution_modified": Resolution_dict})
data = data[(data["Resolution_modified"] != "Drop") & (data["Resolution_modified"] != "In Process")]

### 2 Read and combined Panel data

In [925]:
# TWIST Panel data
TWIST_samples_meta = pd.read_csv(data_location / "PANCAN_TWIST_sample.tsv", sep="\t")
TWIST_pair_meta = pd.read_csv(data_location / "PANCAN_TWIST_pair.tsv", sep="\t")
TWIST_samples = TWIST_samples_meta[["entity:sample_id", "external_id_validation", "participant", "patient_id", "sample_type", "renamed_bai_file", "renamed_bam_file", "depth_of_cov_qc_result", "cnv_calls", "media"]]
TWIST_samples['tn_decision_clean'] = "Not Reported"
TWIST_pair = TWIST_pair_meta[["case_sample", "filtered_variants", "mutect1_callstats", "mutect1_coveragewig", "mutect1_powerwig", "mutect1_vcf", "mutect2_vcf", "oncotated_maf_mutect1", "oncotated_maf_mutect2"]]

TWIST_samples = TWIST_samples[TWIST_samples["entity:sample_id"].isin(TWIST_pair["case_sample"])]
TWIST_samples = TWIST_samples.rename(columns={"entity:sample_id":"case_sample"})
TWIST = pd.merge(TWIST_samples, TWIST_pair, on='case_sample')
TWIST = TWIST.replace(r'^\s*$', np.nan, regex=True)
TWIST["Dataset"] = "TWIST"

<ipython-input-925-dde66bea5383>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  TWIST_samples['tn_decision_clean'] = "Not Reported"


In [926]:
# TSCA Panel data
TSCA_samples_meta = pd.read_csv(data_location / "TSCA_HCMI_sample.tsv", sep="\t")
TSCA_pair_meta = pd.read_csv(data_location / "TSCA_HCMI_pair.tsv", sep="\t")
TSCA_samples = TSCA_samples_meta[["entity:sample_id", "bsp_sample_id_validation", "external_id_validation", "participant", "sample_type", "renamed_bai_file", "renamed_bam_file", "depth_of_cov_qc_result", "cnv_calls", "media", "tn_decision_clean"]]
TSCA_pair = TSCA_pair_meta[["case_sample", "filtered_variants", "mutect1_callstats", "mutect1_coveragewig", "mutect1_powerwig", "mutect1_vcf", "mutect2_vcf", "oncotated_maf_mutect1", "oncotated_maf_mutect2"]]
TSCA_pair = TSCA_pair.dropna()

TSCA_samples = TSCA_samples[TSCA_samples["entity:sample_id"].isin(TSCA_pair["case_sample"])]
TSCA_samples = TSCA_samples.rename(columns={"entity:sample_id":"case_sample"})
TSCA = pd.merge(TSCA_samples, TSCA_pair, on='case_sample')
TSCA = TSCA.drop(columns=["case_sample"])
TSCA = TSCA.rename(columns={"bsp_sample_id_validation":"case_sample"})
TSCA = TSCA.replace(r'^\s*$', np.nan, regex=True)
TSCA["Dataset"] = "TSCA"

In [936]:
# Merge TWIST and TSCA with Tableau_data
Panel = pd.concat([TWIST, TSCA])

#Adding PANEL column head
Panel = Panel.rename(columns={"patient_id":"PT-ID"})
Panel.columns = ["PANEL_" + x if i > 4 else x for i,x in enumerate(Panel.columns)]

Panel["participant"] = [str(x) for x in Panel["participant"]]
Panel["PANEL_tn_decision_clean"] = [str(x) for x in Panel["PANEL_tn_decision_clean"]]

### 3 Read RNA data

In [847]:
# RNA data
RNA_meta = pd.read_csv(data_location / "CCLF_RNA_2_0.tsv", sep="\t")
RNA = RNA_meta[["entity:sample_id", 
                     "bsp_sample_id_rna",  
                     "participant", 
                     "sample_type", 
                     "fastq1", 
                     "fastq2", 
                     "bai_file_rna", 
                     "bam_file_rna", 
                     "fusion_predictions", 
                     "fusion_predictions_abridged", 
                     "rnaseqc_count_metrics", 
                     "rnaseqc_count_outputs", 
                     "rnaseqc_exon_counts", 
                     "rnaseqc_gene_counts", 
                     "rnaseqc_gene_rpkm", 
                     "rsem_genes", 
                     "rsem_isoforms", 
                     "star_bam_file", 
                     "star_bam_index", 
                     "star_chimeric_bam_index", 
                     "star_chimeric_junctions", 
                     "star_junctions", 
                     "star_junctions_pass1", 
                     "star_md_bam_file", 
                     "star_md_bam_index", 
                     "star_read_counts",
                    "star_transcriptome_bam"]]

In [848]:
#Adding RNA column head
RNA.columns = ["RNA_" + x if i > 3 else x for i,x in enumerate(RNA.columns)]

In [873]:
# Removing without SM-ID
total_rows = len(RNA)
RNA["bsp_sample_id_rna"] = [str(x) for x in RNA["bsp_sample_id_rna"]]
RNA["participant"] = [str(x) for x in RNA["participant"]]

### 4 Read WES data

In [177]:
# WES data
WES_samples_meta = pd.read_csv(data_location / "CCLF_WES_sample.tsv", sep="\t")
WES_pair_meta = pd.read_csv(data_location / "CCLF_WES_pair.tsv", sep="\t")
WES_samples = WES_samples_meta[["entity:sample_id", "bsp_sample_id_capture", "external_id_capture", "participant", "sample_type", "formatted_bai_file", "formatted_bam_file", "cnv_calls"]]

# Pair is wierd

### 5 Loop through data samples

In [1155]:
def growth_check(x):
    mapping_dict = {
        'Terminated No Growth': 0,
        'Terminated Unlisted': 0,
        'Terminated Fungus Contamination': 0,
        'Terminated Stopped Growing': 0,
        'Terminated Fungus Contamination,No Growth': 0,
        'Terminated Bacterial Contamination': 0,
        'Terminated Bacterial Contamination,No Growth': 0,
        'Terminated Bacterial Contamination,Fungus Contamination,Sample Quality': 0,
        'Terminated Fibroblast Overgrowth': 0,
        'Terminated Feeder Contamination,No Growth': 0,
        'Verified Normal nan': 0,
        'Verified Normal Fibroblast Overgrowth': 0,
        'Verified Tumor nan': 1,
        'Verified Tumor Stopped Growing': 0,
        'Terminated Feeder Contamination': 0,
        'Verified Tumor Fingerprinting Failure': 2,
        'Verified Tumor Unlisted': 0,
        'Verified Normal Mouse Contamination': 0,
        'Verified Normal Stopped Growing': 0,
        'Verified Normal Unlisted': 0,
        'Verified Normal Sample Purity': 0,
        'Terminated Fingerprinting Failure': 2,
        'Terminated Bacterial Contamination,Fungus Contamination': 0,
        'Terminated Bacterial Contamination,Fungus Contamination,No Growth': 0,
        'Terminated No Growth,Sample Quality': 0,
        'Terminated Sample Quality': 0,
        'Terminated Lab Error,Sample Purity': 0,
        'Terminated Mouse Contamination': 0,
        'Terminated Fibroblast Overgrowth,Mouse Contamination': 0,
        'Verified Tumor No Growth': 0,
        'Terminated No Growth After Thaw': 0,
        'Terminated No Growth,Sample Purity,Sample Quality': 0,
        'Terminated Lab Error': 0,
        'Terminated Fingerprinting Failure,Lab Error': 0,
        'Verified Normal No Growth': 0,
        'Terminated No Growth,Sample Purity': 0,
        'Terminated Sample Purity': 0,
        'Verified Tumor Terminated': 0,
        'Terminated Terminated': 0,
        'Terminated Received/In Container': 0,
        'Completed Terminated': 0,
        'Completed Received/Out of Storage': 0,
        'Terminated nan': 0,
        'nan nan': 2,
        'Verified Tumor Received/Out of Storage': 2,
        'Verified Tumor Received/In Container': 2,
        'Verified Normal Terminated': 0,
        'Verified Normal Received/In Container': 2,
        'Verified Normal Received/Out of Storage': 2,
        'nan Terminated': 0,
        'nan Received/In Container': 2,
        'Terminated Received/Out of Storage': 0,
        'nan Received/Out of Storage': 2,
        'Completed nan': 1,
    }
    
    # Checking resolution and mapping back
    x["Fail Mode"] = [str(x) for x in x["Fail Mode"]]
    
    series = x["Resolution_modified"] + " " + x["Fail Mode"]
    series = [str(x) for x in series]

    x["Growth"] = series
    x = x.replace({"Growth": mapping_dict})

    x = x[x["Growth"] != 2]
    return x

In [1156]:
data = growth_check(data)

In [1292]:
def create_score(x):
    Lookup_table = {
        "PRIMARY":19,
        "P0":18,
        "P1":17,
        "P2":16,
        "P3":15,
        "P4":14,
        "P5":13,
        "P6":12,
        "P7":11,
        "P8":10,
        "P9":9,
        "P10":8,
        "P11":7,
        "P12":6,
        "P13":5,
        "P14":4,
        "P15":3,
        "P16":2,
        "P17":1,
    }

    test = [x.upper().split("_") for x in x["external_id_validation"]]
    
    value = []
    for t in test:
        for key in t:
            if key in Lookup_table:
                value_tmp = Lookup_table[key]
                break
            else:
                value_tmp = 0

        value.append(value_tmp)
        
    x["test"] = value
    
    return x

In [1423]:
empty_numpy_panel = np.empty([1,len(list(Panel)[5:])])
empty_numpy_panel.fill(np.nan)
empty_frame_panel = pd.DataFrame(empty_numpy_panel, columns=list(Panel)[5:])

empty_numpy_RNA = np.empty([1,len(list(RNA)[4:])])
empty_numpy_RNA.fill(np.nan)
empty_frame_RNA = pd.DataFrame(empty_numpy_RNA, columns=list(RNA)[4:])

In [1435]:
def add_nan_panel(data_tmp, empty_numpy_panel):
    data_tmp = pd.concat([data_tmp.reset_index(drop=True), pd.concat([empty_frame_panel]*len(data_tmp)).reset_index(drop=True)], axis=1)
    return(data_tmp)

def add_nan_RNA(data_tmp, empty_frame_RNA):
    data_tmp = pd.concat([data_tmp.reset_index(drop=True), pd.concat([empty_frame_RNA]*len(data_tmp)).reset_index(drop=True)], axis=1)
    return(data_tmp)

In [1508]:
spreadsheet = []

for i, cell_line in enumerate(data["Cell Line"].unique()):
    data_tmp = data[data["Cell Line"] == cell_line]
    # At least one that isn't Verified Normal
    if all(data_tmp["Resolution_modified"] == "Verified Normal") != True:
        # First get the right data from Panel/RNA
        if cell_line.endswith("T"):
            Panel_tmp = Panel[Panel["participant"] == cell_line[:-1]]
            RNA_tmp = RNA[RNA["participant"] == cell_line[:-1]]
        else:
            Panel_tmp = Panel[Panel["participant"] == cell_line]
            RNA_tmp = RNA[RNA["participant"] == cell_line]

        #Add Panel data
        if not Panel_tmp.empty:
            # Check if all are not VN
            if all(Panel_tmp["PANEL_tn_decision_clean"] == "VN") != True:
                # Check if multiple sequencing results for cell line
                if len(Panel_tmp) == 1:
                    Panel_tmp = Panel_tmp.iloc[0:, 5:]
                    data_tmp = pd.concat([data_tmp.reset_index(drop=True), pd.concat([Panel_tmp]*len(data_tmp)).reset_index(drop=True)], axis=1)
                else:
                    # Loop through these panels
                    Panel_tmp = create_score(Panel_tmp)
                    Panel_tmp = Panel_tmp[Panel_tmp["test"] == max(Panel_tmp["test"])]
                    if all(Panel_tmp["PANEL_tn_decision_clean"] == "VN") != True:
                        if len(Panel_tmp) == 1:
                            Panel_tmp = Panel_tmp.iloc[0:1, 5:-1]
                            data_tmp = pd.concat([data_tmp.reset_index(drop=True), pd.concat([Panel_tmp]*len(data_tmp)).reset_index(drop=True)], axis=1)
                        else:
                            # Just picking the one which is not VN
                            Panel_tmp = Panel_tmp[Panel_tmp["PANEL_tn_decision_clean"] != "VN"]
                            if not Panel_tmp.empty:
                                Panel_tmp_tmp = Panel_tmp[Panel_tmp["PANEL_tn_decision_clean"] == "VT"]
                                if not Panel_tmp_tmp.empty:
                                    Panel_tmp_tmp = Panel_tmp_tmp.iloc[0:1, 5:-1]
                                    data_tmp = pd.concat([data_tmp.reset_index(drop=True), pd.concat([Panel_tmp_tmp]*len(data_tmp)).reset_index(drop=True)], axis=1)
                                else:
                                    Panel_tmp = Panel_tmp.iloc[0:1, 5:-1]
                                    data_tmp = pd.concat([data_tmp.reset_index(drop=True), pd.concat([Panel_tmp]*len(data_tmp)).reset_index(drop=True)], axis=1)
                            else:
                                #Add empty columns
                                data_tmp = add_nan_panel(data_tmp, empty_frame_panel)
                                
                    else:
                        #Add empty columns
                        data_tmp = add_nan_panel(data_tmp, empty_frame_panel)

            else:
                #Add empty columns
                data_tmp = add_nan_panel(data_tmp, empty_frame_panel)
        
        else:
            #Add empty columns
            data_tmp = add_nan_panel(data_tmp, empty_frame_panel)
                
        #Add RNA data
        if not RNA_tmp.empty:
            if len(RNA_tmp) == 1:
                RNA_tmp = RNA_tmp.iloc[0:1, 4:]
                data_tmp = pd.concat([data_tmp.reset_index(drop=True), pd.concat([RNA_tmp]*len(data_tmp)).reset_index(drop=True)], axis=1)
            else:
                if len(data_tmp) == 1:
                    RNA_tmp = RNA_tmp.iloc[0:1, 4:]
                    data_tmp = pd.concat([pd.concat([data_tmp]*len(RNA_tmp)).reset_index(drop=True), RNA_tmp.reset_index(drop=True)], axis=1)
                else:
                    # Just pick one
                    RNA_tmp = RNA_tmp.iloc[0:1, 4:]
                    data_tmp = pd.concat([data_tmp.reset_index(drop=True), pd.concat([RNA_tmp]*len(data_tmp)).reset_index(drop=True)], axis=1)
                        
        else:
            data_tmp = add_nan_RNA(data_tmp, empty_frame_RNA)
        
    else:
        #Add empty columns
        data_tmp = add_nan_panel(data_tmp, empty_frame_panel)
        data_tmp = add_nan_RNA(data_tmp, empty_frame_RNA)
        
    data_tmp["RNA_match"] = "Unsure"
    spreadsheet.append(data_tmp)

<ipython-input-1292-c83dd44c1f94>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x["test"] = value


In [1509]:
spreadsheet = pd.concat(spreadsheet)

In [1512]:
#renaming header
spreadsheet.columns = ["_".join(x.split(" ")) for x in spreadsheet.columns]
spreadsheet = spreadsheet.rename(columns={"Cell_Line_Starting_Sample":"entity:Cell_Line_Starting_Sample_id", "Resolution":"Resolution_Original", "Resolution_modified":"Resolution"})

In [1513]:
spreadsheet

,entity:Cell_Line_Starting_Sample_id,PT-ID,SM-ID,Status,Resolution_Original,T/N,Primary_Disease,Tissue_Site,Tumor_Type,Cell_Line,...,RNA_star_bam_index,RNA_star_chimeric_bam_index,RNA_star_chimeric_junctions,RNA_star_junctions,RNA_star_junctions_pass1,RNA_star_md_bam_file,RNA_star_md_bam_index,RNA_star_read_counts,RNA_star_transcriptome_bam,RNA_match
0,SM012T,PT-1PDR8,SM-9RFG2,Closed,Completed,Tumor,invasive ductal carcinoma,NaN,NaN,SM012T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unsure
0,DW036T,NaN,SM-CNRSK,Closed,Completed,Tumor,NaN,NaN,NaN,DW036T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unsure
1,DW036T,PT-21FSL,,Closed,Completed,Tumor,t-pll,NaN,NaN,DW036T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unsure
2,DW036T,NaN,SM-CNRSM,Closed,Terminated,Tumor,NaN,NaN,NaN,DW036T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unsure
3,DW036T,NaN,SM-CNRSJ,Closed,Terminated,Tumor,NaN,NaN,NaN,DW036T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unsure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,SM-J8A8G,PT-2NPEJ,SM-JS9NN,Closed,Verified Tumor,Tumor,medulloblastoma,Brain,Primary,CCLF_KL1338T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unsure
85,SM-J8A8G,PT-2NPEJ,SM-JT22O,Closed,Verified Tumor,Tumor,medulloblastoma,Brain,Primary,CCLF_KL1338T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unsure
86,SM-J8A8G,PT-2NPEJ,SM-JU1FM,Closed,Verified Tumor,Tumor,medulloblastoma,Brain,Primary,CCLF_KL1338T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unsure
87,SM-J8A8G,PT-2NPEJ,SM-JUBKV,Closed,Verified Tumor,Tumor,medulloblastoma,Brain,Primary,CCLF_KL1338T,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unsure


In [1498]:
# Create small spreadsheet if both no RNA and Panel data
spreadsheet["PANEL_renamed_bai_file"] = [str(x) for x in spreadsheet["PANEL_renamed_bai_file"]]
spreadsheet["RNA_fastq1"] = [str(x) for x in spreadsheet["RNA_fastq1"]]
spreadsheet_small = spreadsheet[(spreadsheet["PANEL_renamed_bai_file"] != "nan") | (spreadsheet["RNA_fastq1"] != "nan")]

In [1499]:
spreadsheet_small.to_csv(data_location / "spreadsheet_small.tsv", encoding = 'utf-8', sep='\t')
spreadsheet.to_csv(data_location / "spreadsheet_compleet.tsv", encoding = 'utf-8', sep='\t')